# ATRACKCS: Storm Tracking Pipeline Example

In [1]:
from atrackcs import ATRACKCSConfig
from atrackcs import read_identify_mcs_parallel
from atrackcs import finder_msc
from atrackcs import resume_track 

--------------------------------------------------------------------------
1. SETUP: Define Paths and Configuration
--------------------------------------------------------------------------


In [2]:
# IMPORTANT: Replace these with your actual data and results directories
path_tb = '../test/WA/tb/'
path_p = '../test/WA/p/'
path_results = '../test/Results/'

# Initialize configuration
# You can use the default settings or customize them
my_config = ATRACKCSConfig()

# Example: Customize parameters for stricter filtering
my_config.AREA_VALUE = 2000   # Set minimum area to 3000 km^2
my_config.DURATION = 5        # Set minimum track duration to 8 hours

--------------------------------------------------------------------------
2. Identification (Read Data, Identify Storm Cores)
--------------------------------------------------------------------------


In [3]:
print("--- Starting Storm Identification (Step 1 of 3) ---")

# This function reads the NetCDF files, processes them in parallel, 
# and saves the intermediate GeoDataFrame of polygons to the PATH_RESULTS 
# directory (as 'polygons_cores.pkl').

read_identify_mcs_parallel(path_tb, path_p, path_results, my_config)

--- Starting Storm Identification (Step 1 of 3) ---
Number of Tb files found: 96
[After loading all precipitation] Memory usage: 639.64 MB
Storms identification process completed in 7.4 seconds
Saved polygons to ../test/Results/polygons_cores.pkl
[All done] Memory usage: 657.95 MB


--------------------------------------------------------------------------
3. Tracking (Link Identified Cores over Time)
--------------------------------------------------------------------------


In [6]:
print("\n--- Starting Storm Tracking (Step 2 of 3) ---")

# The finder_msc function loads the intermediate .pkl file, applies the 
# overlap threshold (from config), and returns the GeoDataFrame with track IDs.
tracked_storms = finder_msc(path_results, my_config)


--- Starting Storm Tracking (Step 2 of 3) ---
Estimating trajectories: 


Tracking Polygons: 100%|██████████| 1304/1304 [00:16<00:00, 81.00it/s] 


--------------------------------------------------------------------------
4. Features and Summary (Filter and Save Final Results)
--------------------------------------------------------------------------

In [7]:
print("\n--- Step 3: Summarizing Tracks and Extracting Features ---")
# The resume_track function filters tracks based on the minimum DURATION 
# (e.g., 8 hours), calculates summary features, and saves the final output 
# to a CSV file in PATH_RESULTS.
final_summary = resume_track(tracked_storms, path_results, my_config)



--- Step 3: Summarizing Tracks and Extracting Features ---
Estimating distance and direction between geometric centroids: 


100%|██████████| 237/237 [00:04<00:00, 57.47it/s]


Estimating distance and duration of tracks...


In [8]:
if final_summary is not None:
    # Count the unique track IDs found in the final summary
    num_tracks = len(final_summary.index.get_level_values('track_id').unique())
    
    print("\n==============================================")
    print("ATRACKCS Pipeline Complete!")
    print(f"Total MCSs Found (Duration >= {my_config.DURATION}h): {num_tracks}")
    print(f"Results saved in: {path_results}")
    print("==============================================")


ATRACKCS Pipeline Complete!
Total MCSs Found (Duration >= 5h): 84
Results saved in: ../test/Results/


In [9]:
final_summary

time  \
track_id      polygon_id                                   
06b0c11f-de99 40c3-820e-9864e6c57eaf 2016-08-01 04:00:00   
              4741-850d-88c7f99814e5 2016-08-01 05:00:00   
              4957-9f7d-ed42afa77fd9 2016-08-01 06:00:00   
              4761-8f4b-89cb1e36a46e 2016-08-01 07:00:00   
              4eb7-b281-2a34896257d5 2016-08-01 08:00:00   
...                                                  ...   
f11bee1f-38af 4314-8a1d-bcea422e2c79 2016-08-01 23:00:00   
              4313-85e1-11836ec6621c 2016-08-02 00:00:00   
              4908-a599-7897ad659162 2016-08-02 01:00:00   
              4493-ae0e-3aa9fd2d9379 2016-08-02 02:00:00   
              42e3-8e4f-26cc7eb3eabc 2016-08-02 03:00:00   

                                                                               geometry  \
track_id      polygon_id                                                                  
06b0c11f-de99 40c3-820e-9864e6c57eaf  POLYGON ((-17.05 11.55, -17.05 11.55, -17.05 1...   
              4741-850d-88c7f99814e5  POLYGON ((-17.15 11.25, -17.15 11.25, -17.15 1...   
              4957-9f7d-ed42afa77fd9  POLYGON ((-17.45 12.05, -17.45 12.05, -17.25 1...   
              4761-8f4b-89cb1e36a46e  POLYGON ((-18.25 10.95, -18.25 10.95, -18.25 1...   
              4eb7-b281-2a34896257d5  POLYGON ((-19.85 12.05, -19.85 12.05, -19.75 1...   
...                                                                                 ...   
f11bee1f-38af 4314-8a1d-bcea422e2c79  POLYGON ((18.15 2.05, 18.15 2.05, 18.15 2.45, ...   
              4313-85e1-11836ec6621c  POLYGON ((15.85 4.25, 15.85 4.25, 15.85 4.45, ...   
              4908-a599-7897ad659162  POLYGON ((15.95 3.25, 15.95 3.25, 16.35 4.35, ...   
              4493-ae0e-3aa9fd2d9379  POLYGON ((16.25 3.15, 16.25 3.15, 16.25 3.65, ...   
              42e3-8e4f-26cc7eb3eabc  POLYGON ((16.05 3.45, 16.05 3.45, 17.15 4.25, ...   

                                       area_tb  tb_overshoot  pp_10rate  \
track_id      polygon_id                                                  
06b0c11f-de99 40c3-820e-9864e6c57eaf    4743.2          True       True   
              4741-850d-88c7f99814e5   11643.1          True       True   
              4957-9f7d-ed42afa77fd9   14941.9          True       True   
              4761-8f4b-89cb1e36a46e   27600.4          True       True   
              4eb7-b281-2a34896257d5   73105.8          True       True   
...                                        ...           ...        ...   
f11bee1f-38af 4314-8a1d-bcea422e2c79   42494.5          True       True   
              4313-85e1-11836ec6621c  149813.9          True       True   
              4908-a599-7897ad659162  100236.7          True      False   
              4493-ae0e-3aa9fd2d9379   65739.9          True       True   
              42e3-8e4f-26cc7eb3eabc   62246.7          True       True   

                                          volum_pp  mean_pp  \
track_id      polygon_id                                      
06b0c11f-de99 40c3-820e-9864e6c57eaf  1.148706e+07   2.4218   
              4741-850d-88c7f99814e5  4.438901e+07   3.8125   
              4957-9f7d-ed42afa77fd9  9.766332e+07   6.5362   
              4761-8f4b-89cb1e36a46e  1.149160e+08   4.1636   
              4eb7-b281-2a34896257d5  1.617145e+08   2.2121   
...                                            ...      ...   
f11bee1f-38af 4314-8a1d-bcea422e2c79  2.486672e+07   0.5852   
              4313-85e1-11836ec6621c  1.106238e+08   0.7384   
              4908-a599-7897ad659162  9.336775e+07   0.9315   
              4493-ae0e-3aa9fd2d9379  5.897165e+07   0.8970   
              42e3-8e4f-26cc7eb3eabc  4.992906e+07   0.8021   

                                      mean_pp_above_rate  hvy_prec_frac  \
track_id      polygon_id                                                  
06b0c11f-de99 40c3-820e-9864e6c57eaf              2.4218       0.026028   
              4741-850d-88c7f99814e5              3.8125       0.0